# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 download your data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [4]:
df = pd.read_excel('raw_data/2023.xlsx')
df.to_csv('raw_data/2023.csv', index=False)
pd.set_option('display.max_colwidth', None)
df.sample(10)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments
34512,2023-09-06 12:51:00,ES274,915947,ESC: ES274 - 476 - 86 St - Island Plat,This escalator is out of service due to LIFTNET event: Stopped,BRP,Closed,Stopped,REPAIR,Repair,2023-09-06 21:00:00,2023-09-06 13:30:00,ST230906.txt - 2332,External Monitoring System,EE-LNOUT,2023-09-06 13:03:00,CL,NaN
19357,2023-05-21 19:15:00,EL403,873589,ELE: EL403 - 222 - Roosevelt Island - Brooklyn Bound Plat,NaN,ENT,Closed,NaN,REPAIR,Repair,2023-05-23 02:00:00,2023-05-22 12:15:00,NaN,Phone,EE-MANOUT,2023-05-21 19:19:00,CL,NaN
39116,2023-10-09 16:57:00,ES254X,930160,ESC: ES254X - 275 - Lexington Av/53 St - Island Plat,This Escalator is out service reported by Mow #99,TPE,Closed,NaN,REPAIR,Repair,2023-10-15 17:00:00,2023-10-13 11:24:00,NaN,Phone,EE-MANOUT,2023-10-09 16:59:00,CL,This Escalator is out service reported by Mow #99\nEmail sent to Station district 2 Sent email to Dist. Field Office for\nan update. ERTS changed to 10/12/23. As per email from S/S Faller @ 06:22 : ES254X -\nLexington Av/53 St - Island Plat - OOS Notified Gvaramiya to check machine\nstatus. ERTS changed to 10/13/23. Notified Gvaramiya to check machine status. ERTS changed to 10/14/23. Notified Gvaramiya to check machine status. ERTS changed to 10/15/23. As per Charley Cohen this escalator has been returned to service.
16255,2023-04-30 01:24:00,ES334,838769,ESC: ES334 - 103 - Bowery - Mezz A,NaN,CLN,Closed,NaN,CLEANING,Cleaning,2023-04-30 07:00:00,2023-04-30 02:47:00,NaN,Phone,EE-MANOUT,2023-04-30 01:24:00,CL,***This Escalator is out of service for Cleaning***\n\n\n CTA Ramnarine #727589 Reports stopped machine for cleaning. 4/30/23 @ 0247 (Rosario) CTA Ramnarine #727589 Reports stopped machine for cleaning. RTS\n\n
3467,2023-01-26 18:45:00,EL732,801933,ELE: EL732 - 332 - Fulton St - Island Plat,NaN,BRP,Closed,NaN,REPAIR,Repair,2023-01-27 04:00:00,2023-01-26 21:50:00,NaN,E-mail,EE-MANOUT,2023-01-26 18:57:00,CL,NaN
10361,2023-03-20 20:30:00,ES345,821979,ESC: ES345 - 414 - Bowling Green - Outside Area,NaN,BRP,Closed,Not Running,REPAIR,Repair,2023-03-21 05:00:00,2023-03-21 00:55:00,ST230320.txt - 3717,External Monitoring System,EE-LNOUT,2023-03-20 20:46:00,CL,NaN
8001,2023-03-03 00:19:00,ES457,815506,ESC: ES457 - 447 - Flushing-Main St - Mezz B,NaN,CLN,Closed,NaN,CLEANING,Cleaning,2023-03-03 08:00:00,2023-03-03 03:13:00,NaN,Phone,EE-MANOUT,2023-03-03 00:20:00,CL,\n\n*** This escalator is out of service for Routine Cleaning ***
31476,2023-08-14 16:26:00,ES414,907159,ESC: ES414 - 222 - Roosevelt Island - Mezz B,NaN,BRP,Closed,Lwr Level Step,REPAIR,Repair,2023-08-16 23:00:00,2023-08-15 00:10:00,ST230814.txt - 3056,External Monitoring System,EE-LNOUT,2023-08-14 16:39:00,CL,NaN
49750,2023-12-27 06:40:00,ES453,960459,ESC: ES453 - 267 - Jackson Hts-Roosevest Av - Mezz A,This escalator is out of service due to LIFTNET event: Lower Comb Impact Switches,VAN,Closed,Lower Comb Impact Switches,UNDERINVESTG,Under Investigation,2023-12-27 15:00:00,2023-12-27 07:50:00,ST231227.txt - 764,External Monitoring System,EE-LNOUT,2023-12-27 06:42:00,CL,NaN
30205,2023-08-05 04:08:00,ES114,903545,ESC: ES114 - 219 - 161 St-Yankee Stadium - Mezz B,NaN,IRP,Closed,NaN,REPAIR,Repair,2023-08-07 06:00:00,2023-08-05 13:15:00,NaN,Phone,EE-MANOUT,2023-08-05 04:12:00,CL,NaN


In [5]:
%%R
df <- read_csv('raw_data/2023.csv')

Rows: 50296 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): Common Name, Equipment Description, Executive Comment, Outage Cod...
dbl   (1): Outage
dttm  (4): Out of Service Date, Estimated Return to Service Date, Actual Ret...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## 👉 convert addresses --> lat/long 

See the [census-examples](https://github.com/data4news/census-examples) repository for examples. If you need help, try asking in the class slack channel. Chances are someone in the class is struggling with the same problem as you are so we might as well all learn together in the same slack channel! 

In [6]:
# filter down to elevators only
df_ele = df[df['Equipment Description'].str.startswith('ELE')]
df_ele.sample(10)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments
9726,2023-03-16 09:28:00,EL215,820384,ELE: EL215 - 318 - 34 St-Penn Station - Island Plat,NaN,APC,Closed,NaN,PLANNEDWORK,Planned Work,2023-03-16 16:00:00,2023-03-16 10:25:00,NaN,Phone,EE-MANOUT,2023-03-16 09:28:00,CL,***This elevator is out of service A&P for Sanska contractors for fire alarm testing***\n\nE&EM Friday report OOS A&P for Sanska contractors for fire alarm testing. WO # 6062015. E&EM Friday completed A&P for Sanska contractors. BIS@1025
27133,2023-07-14 00:26:00,EL119,895007,ELE: EL119 - 146 - 181 St - Mezzanine A at 184 St,NaN,INS,Closed,NaN,INSPECTION,Inspection,2023-07-14 07:00:00,2023-07-14 04:05:00,NaN,Phone,EE-MANOUT,2023-07-14 00:26:00,CL,"***This Elevator is out of service for Inspection***\n\n\nE&EM Walsh, E&EHs Hunte and Vezzuso arrived for CAT1 inspection.Third party witness Raymond Camacho on site. WO #6596750 Machine passed inspection RTS @0405HRS."
48336,2023-12-17 00:00:00,EL109,956298,ELE: EL109 - 301 - 181 St,This Elevator is out for Cat 1 inspection.,INS,Closed,NaN,INSPECTION,Inspection,2023-12-17 08:00:00,2023-12-17 04:00:00,NaN,Phone,EE-MANOUT,2023-12-17 00:06:00,CL,E&EMs Hernández/ Mendez and TEA Whistler on site at El109 at 11:50. Here for CAT 1 inspection OOS at 00:00. WO#8276095 Completed CAT 1 inspection. Passed CAT 1 inspection. Test ran BIS at 0400
41094,2023-10-25 00:16:00,EL771,935793,ELE: EL771 - 135 - Livonia Av - Brooklyn Bound,This elevator is out of service due to LIFTNET fault: Door Opening/ F. Door Open Limit failure,BRP,Closed,Door Opening/ F. Door Open Limit failure,REPAIR,Repair,2023-10-25 09:00:00,2023-10-25 02:00:00,ST231025.txt - 43,External Monitoring System,EE-LNOUT,2023-10-25 00:28:00,CL,NaN
34418,2023-09-05 23:18:00,EL112,915622,ELE: EL112 - 302 - 168 St,This elevator is out of service for Monthly PM,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-09-06 07:00:00,2023-09-06 04:30:00,NaN,Phone,EE-MANOUT,2023-09-05 23:19:00,CL,"WO7751068 EAM OUT915622\n9/05/23 2318hrs {Jackson,R}E&EM's Whitaker, Sampson and TEA Figaro taking unit OOS for Monthly PM Completed maintenance, Test Ran BIS at 0430."
1472,2023-01-12 00:45:00,EL738,795438,ELE: EL738 - 73 - New Utrecht - Location TBD,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-01-12 09:00:00,2023-01-12 05:30:00,NaN,Phone,EE-MANOUT,2023-01-12 00:46:00,CL,"***This elevator is out of service for Quarterly PM***\nWO5408078 EAM OUT795438\n1/12/23 0045hrs {Jackson,R}E&EM Dhuman and E&EM Furmanov taking unit OOS for Quarterly PM Completed quarterly maintenance. Found Kellem grip broken and traveler cable has a cut through it. Report to MS1 Ivanov. Test ran and monitored. B.I.S@ 05:30 as per MS1 Ivanov"
12008,2023-03-30 23:30:00,EL442,827122,ELE: EL442 - 223 - Lexington Av/63 St,NaN,MRP,Closed,NaN,PLANNEDWORK,Planned Work,2023-03-31 08:00:00,2023-03-31 03:00:00,NaN,Phone,EE-MANOUT,2023-03-30 23:33:00,CL,"***The elevator is out of service due to pit lights not working***\n\nWork Order3425974\n\nE&EM Zamora/E&EH Grzybowski here for defects @2330. OOS time: 2330 hours E&EM Zamora and E&EH Grzybowski found escape hatch secure.\nCreated: 03/31/2023 05:22\nGrzybowski, Joseph (1272853)\n \nE&EM Zamora/E&EH Grzybowski here for defects @2330\t\nCreated: 03/30/2023 23:30\nZamora, Deamiah (1252821)\nUpon arrival, changed one light bulb in pit.\t\nCreated: 03/31/2023 05:23\nGrzybowski, Joseph (1272853)"
5514,2023-02-11 15:04:00,EL708,807998,ELE: EL708 - 174 - Jay St-Metro Tech - Brooklyn Bound Plat,NaN,FR,Cancelled,DOOR OPENING / BOTTOM RESTRICTOR FAILURE,UNDERINVESTG,Under Investigation,2023-02-12 00:00:00,NaT,ST230211.txt - 2376,External Monitoring System,EE-LNOUT,2023-02-11 15:18:00,C,As per Legacy Server Elevator is running
37815,2023-09-29 

In [7]:
# create column `station MRN` to merge with `Station ID` to geolocate
df_ele['Station MRN'] = df_ele['Equipment Description'].str.extract(r'ELE: EL\d{2,3}\w? - (\d{2,3})\s-')

In [8]:
# check what the are the missing station MRN
# HOW DO I GET THESE!!!

df_ele[df_ele['Station MRN'].isna()].head(5)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN
413,2023-01-04 07:23:00,EL740,792397,ELE: EL740 - 73 - New Utrecht - Location TBD,NaN,INS,Closed,NaN,INSPECTION,Inspection,2023-01-04 16:00:00,2023-01-04 11:45:00,NaN,Phone,EE-MANOUT,2023-01-04 07:24:00,CL,"***This elevator is out of service for CAT1 Inspection***\n\n\nE&EM Stewart and E&EH Crawford taking unit OOS for ASME CAT1 inspection. WO#5408464 (EAM) E&EH Crawford: ASME CAT1 inspection completed, machine passed inspection. Test ran RTS @1145",NaN
450,2023-01-04 11:45:00,EL741,792554,ELE: EL741 - 73 - New Utrecht - Location TBD,NaN,INS,Closed,NaN,INSPECTION,Inspection,2023-01-04 17:00:00,2023-01-04 13:15:00,NaN,Phone,EE-MANOUT,2023-01-04 11:48:00,CL,"***This unit was removed from service by E&E personnel to perform inspection***\nWO 5408465\nE&EM Stewart and E&EH Crawford here for ASME CAT1 inspection.\n E&EH Crawford report: ASME CAT1 inspection completed, machine passed inspection. Test ran RTS @1315.",NaN
595,2023-01-05 11:20:00,EL739,792984,ELE: EL739 - 73 - New Utrecht - Location TBD,NaN,INS,Closed,NaN,INSPECTION,Inspection,2023-01-05 17:00:00,2023-01-05 13:20:00,NaN,Phone,EE-MANOUT,2023-01-05 11:23:00,CL,"***This unit was removed from service by E&E personnel to perform inspection***\nWO5408463\nE&EM Stewart and E&EH Crawford here for ASME CAT1 inspection.\n (EAM) E&EM Crawford: ASME CAT1 inspection completed, machine passed inspection. Test ran RTS @1320",NaN
1348,2023-01-11 08:37:00,EL739,795146,ELE: EL739 - 73 - New Utrecht - Location TBD,NaN,ENV,Closed,Door Opening/ F. Door Open Limit failure,REPAIR,Repair,2023-01-11 17:00:00,2023-01-11 11:30:00,ST230111.txt - 1256,External Monitoring System,EE-LNOUT,2023-01-11 08:44:00,CL,NaN,NaN
1818,2023-01-14 09:10:00,EL156,796470,ELE: EL156 - TBD - Dyckman St Quarters - Location TBD,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-01-14 18:00:00,2023-01-14 12:20:00,NaN,Phone,EE-MANOUT,2023-01-14 09:13:00,CL,***This elevator is out of service for preventive maintenance***\n\n\n\n\n\nE&EMs Siu/Diaz taking unit OOS for bimonthly PM. WO#5407838 bimonthly PM. completed BIS,NaN


In [9]:
# geocoding my dataset: merging stations by `Station ID` and `Station MRN`

# convert string to float
df_ele['Station MRN'] = df_ele['Station MRN'].astype('Int64')

# stations with MRN > 523 are not regular subway stations and not geolocatable
# drop all stations MRN > 523 and save to a new file
df_ele[df_ele['Station MRN'] <= 523].to_csv('data/2023_ele.csv', index=False)

# filter down to geolocatable columns
df_ele = df_ele[df_ele['Station MRN'] <= 523]
df_ele.sample(5)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN
16038,2023-04-29 01:36:00,EL371,838277,ELE: EL371 - 026 - Dekalb Av - Manhattan Bound Plat,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-04-29 09:00:00,2023-04-29 03:00:00,NaN,Phone,EE-MANOUT,2023-04-29 01:37:00,CL,\n\n***This elevator is out of service for Access & Protection for EMD***\n\nE&EH Tsang here for A&P for EMD\n\n\nWO 6378631 EMD completed annual inspection. Cleaned and tested smoke detectors. BIS 0300,26
7456,2023-02-26 17:38:00,EL708,813950,ELE: EL708 - 174 - Jay St-Metro Tech - Brooklyn Bound Plat,NaN,BRP,Closed,NaN,UNDERINVESTG,Under Investigation,2023-02-27 01:00:00,2023-02-26 20:30:00,NaN,Phone,EE-MANOUT,2023-02-26 17:40:00,CL,NaN,174
49683,2023-12-26 20:39:00,EL396,960238,ELE: EL396 - 112 - Myrtle-Wyckoff Avs - Mezz A,This Elevator is out service reported by S/A Gordilo pass# 329090,BRP,Closed,NaN,REPAIR,Repair,2023-12-27 05:00:00,2023-12-27 00:30:00,NaN,Phone,EE-MANOUT,2023-12-26 20:41:00,CL,NaN,112
22019,2023-06-09 14:24:00,EL234,880954,ELE: EL234 - 225 -47-50 Sts-Rockefeller Ctr - Outside Area,NaN,FR,Cancelled,DOWN SLOWDOWN LIMIT OPEN,UNDERINVESTG,Under Investigation,2023-06-09 23:00:00,NaT,ST230609.txt - 2000,External Monitoring System,EE-LNOUT,2023-06-09 14:34:00,C,As per the LIFTNET Galaxy this elevator is in service.,225
31089,2023-08-11 07:10:00,EL110,906205,ELE: EL110 - 301 - 181 St,NaN,FR,Cancelled,Up Normal Limit Failure,UNDERINVESTG,Under Investigation,2023-08-11 16:00:00,NaT,ST230811.txt - 1083,External Monitoring System,EE-LNOUT,2023-08-11 07:22:00,C,As per LIFTNET Galaxy client this elevator is in service.,301


In [11]:
# load the geostation file
df_geostation = pd.read_csv('raw_data/subway_stations.csv')

# merge the geostation file with the df_ele
df_ele_merged = df_ele.merge(df_geostation, left_on='Station MRN', right_on='Station ID')

pd.set_option('display.max_columns', 500)

# rename the columns
df_ele_merged = df_ele_merged.rename(columns={'GTFS Latitude': 'lat',  'GTFS Longitude': 'long'})


In [12]:
# check the merged file 

# save the merged file
df_ele_merged.to_csv('data/2023_ele_geocode.csv', index=False)
print(df_ele_merged.shape)
df_ele_merged.sample(5)


# WHY IS THERE MORE ROWS THAN THE ORIGINAL FILE??

(18208, 35)


,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference
14444,2023-10-17 12:23:00,EL719,932994,ELE: EL719 - 106 - Fulton St - Upper Plat,"This Elevator is out service reported by Vasquez, Edward",BRP,Closed,NaN,REPAIR,Repair,2023-10-17 21:00:00,2023-10-17 13:15:00,NaN,Phone,EE-MANOUT,2023-10-17 12:26:00,CL,NaN,106,106,628,M22,BMT,Jamaica,Fulton St,M,J Z,Subway,40.710374,-74.007582,Brooklyn,Broad St,1,NaN,POINT (-74.007582 40.710374)
1174,2023-01-24 03:39:00,EL441,800456,ELE: EL441 - 223 - Lexington Av/63 St,NaN,UNI,Cancelled,Fire Phase 1,UNDERINVESTG,Under Investigation,2023-01-24 12:00:00,NaT,ST230124.txt - 778,External Monitoring System,EE-LNOUT,2023-01-24 03:51:00,C,Machine was on fire service due to entrapment on EL442,223,223,223,B08,IND,63rd St,Lexington Av/63 St,M,F Q,Subway,40.764629,-73.966113,Uptown - Queens,Downtown & Brooklyn,1,NaN,POINT (-73.966113 40.764629)
3192,2023-03-07 23:41:00,EL601,817144,ELE: EL601 - 471 - 34 St-Hudson Yards - Mezz A,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-03-08 08:00:00,2023-03-08 00:00:00,NaN,Phone,EE-MANOUT,2023-03-07 23:42:00,CL,"***This elevator is out of service for Acces & Protection for Safety***\nEAM OUT817144\n3/07/23 2341hrs {Jackson,R}E&EM Dean and E&EH Ahmed taking unit OOS for A&P for Safety while working on el602 finish fireman service unit BIS @ 0000hrs",471,471,471,726,IRT,Flushing,34 St-Hudson Yards,M,7,Subway,40.755882,-74.001910,Queens,NaN,1,NaN,POINT (-74.00191 40.755882)
9640,2023-07-10 09:50:00,EL341,893159,ELE: EL341 - 188 - Euclid Av - Outside Area,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-07-10 18:00:00,2023-07-10 10:15:00,NaN,Phone,EE-MANOUT,2023-07-10 09:52:00,CL,***This Escalator is out for A&P for EMD Inspection***\n\n\n07/10/23 E&EM Liu47 arrived@0830hrs. A&P for EMD.\n\nE&EM Rodriguez arrived at EL341. Here to provide A&P for EMD\n\nWO#6548153 E&EM Rodriguez report: EMD repaired 1 smoke detector in the pit. RTS @10:15hrs.,188,188,188,A55,IND,8th Av - Fulton St,Euclid Av,Bk,A C,Subway,40.675377,-73.872106,Manhattan,Lefferts - Rockaways,1,NaN,POINT (-73.872106 40.675377)
7102,2023-05-23 16:47:00,EL127,874540,ELE: EL127 - 430 - Simpson St,NaN,VAN,Closed,Main Line Disconnect,UNDERINVESTG,Under Investigation,2023-05-24 01:00:00,2023-05-23 19:10:00,ST230523.txt - 3200,External Monitoring System,EE-LNOUT,2023-05-23 16:56:00,CL,NaN,430,430,430,217,IRT,Lenox - White Plains Rd,Simpson St,Bx,2 5,Elevated,40.824073,-73.893064,Wakefield - Eastchester,Manhattan,1,NaN,POINT (-73.893064 40.824073)


## 👉 convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

In [13]:
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df_ele_merged['lat']
        longitudes = df_ele_merged['long']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df_ele_merged)))

    return pd.DataFrame(data)

census_geos_df = bulk_geocode(df_ele_merged['lat'], df_ele_merged['long']) 
census_geos_df.head()

  0%|          | 0/18208 [00:00<?, ?it/s]

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,INTPTLAT,FUNCSTAT,NAME,OBJECTID,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,610,360470296002002,+40.6008553,2002,0,36,2002,210701004650915,BK,+40.6008553,S,Block 2002,4615836,029600,-073.9943207,2,20213,242,-073.9943207,G5040,L,U,047
1,,0,360050403021003,+40.8669415,1003,0,36,1003,210701006017414,BK,+40.8669415,S,Block 1003,1475683,040302,-073.8935996,1,3799,0,-073.8935996,G5040,L,U,005
2,,593,360610034004000,+40.7305461,4000,0,36,4000,210701008621623,BK,+40.7305461,S,Block 4000,2844262,003400,-073.9817046,4,19330,415,-073.9817046,G5040,L,U,061
3,,0,360810208001000,+40.6998869,1000,0,36,1000,210701006104178,BK,+40.6998869,S,Block 1000,1848413,020800,-073.8089158,1,13876,0,-073.8089158,G5040,L,U,081
4,,0,360810033011019,+40.7492525,1019,0,36,1019,210701006117049,BK,+40.7492525,S,Block 1019,1404062,003301,-073.9374018,1,3969,0,-073.9374018,G5040,L,U,081


In [14]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
census_geos_df = census_geos_df[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,360470296002002,36,047,029600,2002
1,360050403021003,36,005,040302,1003
2,360610034004000,36,061,003400,4000
3,360810208001000,36,081,020800,1000
4,360810033011019,36,081,003301,1019
...,...,...,...,...,...
18203,360610137003000,36,061,013700,3000
18204,360610137003000,36,061,013700,3000
18205,360610137003000,36,061,013700,3000
18206,360050224031001,36,005,022403,1001


## 👉 Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).

In [16]:
df_with_geos = pd.concat(
    [ 
        df_ele_merged.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference,GEOID,STATE,COUNTY,TRACT,BLOCK
0,2023-01-01 00:14:00,EL376,791238,ELE: EL376 - 068 - Bay Pkwy - Outside Area,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-01-01 07:00:00,2023-01-01 05:30:00,NaN,Phone,EE-MANOUT,2023-01-01 00:14:00,CL,***This Elevator is out of service for Maintenance***\n\n\nE&EM R.Brown and E&Eh K.Singh arrived for maintenance at 00:05. WO #5407916 PM Completed and RTS @5.30,68,68,68,B21,BMT,West End,Bay Pkwy,Bk,D,Elevated,40.601875,-73.993728,Manhattan,Coney Island,1,NaN,POINT (-73.993728 40.601875),360470296002002,36,047,029600,2002
1,2023-01-01 01:04:00,EL189,791257,ELE: EL189 - 212 - Kingsbridge Rd - Outside Area,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-01-01 09:00:00,2023-01-01 01:18:00,NaN,Phone,EE-MANOUT,2023-01-01 01:05:00,CL,\n\n***This elevator is out of service for Access & Protection for Safety***,212,212,212,D04,IND,Concourse,Kingsbridge Rd,Bx,B D,Subway,40.866978,-73.893509,Bedford Pk Blvd & 205 St,Manhattan,1,NaN,POINT (-73.893509 40.866978),360050403021003,36,005,040302,1003
2,2023-01-01 02:01:00,EL293,791274,ELE: EL293 - 119 - 1 Av - Brooklyn Bound Platform,NaN,EUF,Closed,UPS Battery Failure,REPAIR,Repair,2023-01-01 11:00:00,2023-01-01 04:50:00,ST230101.txt - 192,External Monitoring System,EE-LNOUT,2023-01-01 02:08:00,CL,NaN,119,119,119,L06,BMT,Canarsie,1 Av,M,L,Subway,40.730953,-73.981628,8 Av,Brooklyn,1,NaN,POINT (-73.981628 40.730953),360610034004000,36,061,003400,4000
3,2023-01-01 02:08:00,EL449X,791275,ELE: EL449X - 279 - Sutphin Blvd-Archer Av - Mezz,NaN,TPE,Closed,NaN,REPAIR,Repair,2023-01-05 14:00:00,2023-01-05 03:06:00,NaN,Phone,EE-MANOUT,2023-01-01 02:09:00,CL,"***This elevator is out of service as reported by SS Patel010647***\n**This machine is out of service for required repairs. NYC Transit is not responsible for the maintenance or repair of this unit**\n At 22:45 on 01/02/2023 MOW#85 states customers entrapped in EL449x at Sutphin blvd. No medical requested. Notified MS1 Pukhalski MS1 Pukhalskiy updated by email Checked the EL449X, nobody inside. Sent email to Dist. Field Office for an update. ERTS changed to 01/05/23.\n\n\n S/S Patel reports found machine running while on station. RTS",279,279,279,G06,IND,Queens - Archer,Sutphin Blvd-Archer Av-JFK Airport,Q,E J Z,Subway,40.700486,-73.807969,Jamaica Center,Manhattan,1,NaN,POINT (-73.807969 40.700486),360810208001000,36,081,020800,1000
4,2023-01-01 05:43:00,EL428,791299,ELE: EL428 - 273 - Queens Plaza - Outside Area,NaN,VAN,Closed,NaN,REPAIR,Repair,2023-01-01 13:00:00,2023-01-01 09:10:00,NaN,Phone,EE-MANOUT,2023-01-01 05:44:00,CL,NaN,273,273,273,G21,IND,Queens Blvd,Queens Plaza,Q,E M R,Subway,40.748973,-73.937243,Forest Hills - Jamaica,Manhattan,1,NaN,POINT (-73.937243 40.748973),360810033011019,36,081,003301,1019


In [17]:
df_with_geos.to_csv('data/2023_subway_censusgeo.csv', index=False)